In [27]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# MPC import
import numpy as np
from LinearMPC.MPCVelControl import MPCVelControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [97]:
Ts = 0.05
sim_time = 10
H = 7
x0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])   # initial state
x_target = np.array([0, 0, 0, 0, 0, 0, 4, 2, 2, 0, 0, 0])   # target state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, x_target=x_target, method="linear"
)

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(
    t_cl[:-1],
    x_cl[:, :-1],
    u_cl,
    Ref=x_target.reshape(-1, 1).repeat(u_cl.shape[1], axis=1),
    T_ol=t_ol[..., :-1],
    X_ol=x_ol,
    U_ol=u_ol,
);

x_var befor =  None
x_var after =  9.75710704715711e-17
SS status: optimal r: [0 0]

x_var befor =  [[ 9.75710705e-17  8.51172981e-01  1.70234596e+00  9.08045031e-01
   5.68720497e-02 -5.68720497e-02  5.68720497e-02 -5.68720497e-02
   5.68720497e-02 -5.68720497e-02  5.68720497e-02 -5.68720497e-02
   5.68720497e-02 -5.68720497e-02  5.68720497e-02 -5.68720497e-02
   5.68720497e-02 -5.68720497e-02  5.68720497e-02 -5.68720497e-02
   5.68720497e-02 -5.68720497e-02  5.68720497e-02 -5.68720497e-02
   5.68720497e-02 -5.68720497e-02  5.68720497e-02 -5.68720498e-02
  -1.50680317e-01 -1.61993436e-01 -1.54760484e-01 -1.47477340e-01
  -1.40796153e-01 -1.34473570e-01 -1.28463951e-01 -1.22754017e-01
  -1.17330971e-01 -1.12182176e-01 -1.07295590e-01 -1.02659791e-01
  -9.82639449e-02 -9.40977757e-02 -9.01515461e-02 -8.64160322e-02
  -8.28825029e-02 -7.95426990e-02 -7.63888144e-02 -7.34134771e-02
  -7.06097330e-02 -6.79710285e-02 -6.54911961e-02 -6.31644397e-02
  -6.09853207e-02 -5.89487457e-02 -5.70499

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=199, step=2), IntSlider(value=0…

### OL XVel target

In [63]:
Ts = 0.05
t0 = 0
x0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0]) 
rocket = Rocket(Ts,model_params_filepath=rocket_params_path)
H = 5 # MPC horizon in seconds
mpc_ol = MPCVelControl().new_controller(rocket, Ts, H)
u, x_ol, u_ol, t_ol = mpc_ol.get_u(t0, x0)

xs =  [0. 0. 0.]
xs =  [0. 0. 0.]
SS status: optimal r: [0. 0.]


### Subsystem velX 

In [96]:
from LinearMPC.MPCControl_xvel import MPCControl_xvel
import cvxpy as cp
Ts = 0.05
sim_time = 30
H = 7
x_target = np.array([0,0,5])
# Use a state closer to equilibrium (with small velocities)
x0 = np.array([0, 0, 0])   # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
xs, us = rocket.trim()

A, B = rocket.linearize(xs, us)

mpc_x = MPCControl_xvel(A, B, xs, us, Ts, H) # Full 12x12 A and 12x4 B matrices
u0, x_traj, u_traj = mpc_x.get_u(x0,x_target)

print("u0 = ",u0)
#print("x_traj = ",x_traj)
#print("u_traj = ",u_traj)

x_var befor =  None
x_var after =  2.613743415794561e-16
u0 =  [-0.26]


### Subsystem velY


In [60]:
from LinearMPC.MPCControl_yvel import MPCControl_yvel
import cvxpy as cp
Ts = 0.05
sim_time = 30
H = 7
x_target = np.array([0,0,5])
# Use a state closer to equilibrium (with small velocities)
x0 = np.array([0, 0, 0])   # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
xs, us = rocket.trim()

A, B = rocket.linearize(xs, us)

mpc_y = MPCControl_yvel(A, B, xs, us, Ts, H) # Full 12x12 A and 12x4 B matrices
u0, x_traj, u_traj = mpc_y.get_u(x0,x_target)

print("u0 = ",u0)
#print("x_traj = ",x_traj)
#print("u_traj = ",u_traj)

xs =  [0. 0. 0.]
u0 =  [0.26]


### Subsystem velZ

In [62]:
from LinearMPC.MPCControl_zvel import MPCControl_zvel
import cvxpy as cp
Ts = 0.05
sim_time = 30
H = 7
x_target = np.array([5])
# Use a state closer to equilibrium (with small velocities)
x0 = np.array([0])   # initial state

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
xs, us = rocket.trim()

A, B = rocket.linearize(xs, us)

mpc_z = MPCControl_zvel(A, B, xs, us, Ts, H) # Full 12x12 A and 12x4 B matrices
u0, x_traj, u_traj = mpc_z.get_u(x0,x_target)

print("u0 = ",u0)
#print("x_traj = ",x_traj)
#print("u_traj = ",u_traj)

u0 =  [80.]
